In [66]:
import os.path as op
from scipy.io.idl import readsav
import pandas as pd
import numpy as np

In [116]:
filedir = '/Users/bryna/Documents/Physics/data_files/fhd_vis_data/'
files = ['1061321792_vis_XX.sav', '1061321792_vis_YY.sav', '1061321792_flags.sav', '1061321792_params.sav']
filelist = []

In [149]:
for f in files:
    filelist.append(op.join(filedir, f))
datafiles = {}
for file in filelist:
    if file.lower().endswith('_vis_xx.sav'):
        datafiles['xx'] = xx_datafile = file
    elif file.lower().endswith('_vis_yy.sav'):
        datafiles['yy'] = yy_datafile = file
    elif file.lower().endswith('_vis_xy.sav'):
        datafiles['xy'] = xy_datafile = file
    elif file.lower().endswith('_vis_yx.sav'):
        datafiles['yx'] = yx_datafile = file
    elif file.lower().endswith('_params.sav'):
        params_file = file
    elif file.lower().endswith('_flags.sav'):
        flags_file = file
    else:
        print(file + ' is not a recognized fhd file type')

vis_data = {}
for pol, file in datafiles.items():
    this_dict = readsav(file, python_dict=True)
    vis_data[pol] = this_dict['vis_ptr']
    this_obs = pd.DataFrame(this_dict['obs'])

obs = this_obs
bl_info = pd.DataFrame(obs['BASELINE_INFO'][0])
meta_data = pd.DataFrame(obs['META_DATA'][0])
astrometry = pd.DataFrame(obs['ASTR'][0])
fhd_pol_list = []
for pol in obs['POL_NAMES'][0]:
    fhd_pol_list.append(pol.decode("utf-8").lower())

params_dict = readsav(params_file, python_dict=True)
params = pd.DataFrame(params_dict['params'])

flags_dict = readsav(flags_file, python_dict=True)
flag_data = {}
for index, f in enumerate(flags_dict['flag_arr']):
    flag_data[fhd_pol_list[index]] = f

In [30]:
obs.columns

Index(['CODE_VERSION', 'INSTRUMENT', 'OBSNAME', 'DIMENSION', 'ELEMENTS',
       'NBASELINES', 'DFT_THRESHOLD', 'KPIX', 'DEGPIX', 'OBSAZ', 'OBSALT',
       'OBSRA', 'OBSDEC', 'ZENRA', 'ZENDEC', 'OBSX', 'OBSY', 'ZENX', 'ZENY',
       'PHASERA', 'PHASEDEC', 'ORIG_PHASERA', 'ORIG_PHASEDEC', 'N_POL',
       'N_TILE', 'N_TILE_FLAG', 'N_FREQ', 'N_FREQ_FLAG', 'N_TIME',
       'N_TIME_FLAG', 'N_VIS', 'N_VIS_IN', 'N_VIS_RAW', 'NF_VIS',
       'BEAM_INTEGRAL', 'POL_NAMES', 'JD0', 'MAX_BASELINE', 'MIN_BASELINE',
       'DELAYS', 'LON', 'LAT', 'ALT', 'FREQ_CENTER', 'FREQ_RES', 'TIME_RES',
       'ASTR', 'ALPHA', 'PFLAG', 'CAL', 'RESIDUAL', 'VIS_NOISE',
       'BASELINE_INFO', 'META_DATA', 'META_HDR', 'DEGRID_SPECTRAL_TERMS',
       'GRID_SPECTRAL_TERMS', 'GRID_INFO', 'HEALPIX'],
      dtype='object')

In [177]:
meta_data.columns

Index(['INPUT', 'ANTENNA', 'TILE', 'POL', 'RX', 'SLOT', 'FLAG', 'LENGTH',
       'NORTH', 'EAST', 'HEIGHT', 'GAINS', 'DELAYS'],
      dtype='object')

In [184]:
astrometry.columns

Index(['NAXIS', 'CD', 'CDELT', 'CRPIX', 'CRVAL', 'CTYPE', 'LONGPOLE',
       'LATPOLE', 'PV2', 'PV1', 'AXES', 'REVERSE', 'COORD_SYS', 'PROJECTION',
       'KNOWN', 'RADECSYS', 'EQUINOX', 'DATEOBS', 'MJDOBS', 'X0Y0'],
      dtype='object')

In [205]:
astrometry['X0Y0']

0    [0.0, 0.0]
Name: X0Y0, dtype: object

In [206]:
Ntimes = obs['N_TIME']
Nbls = obs['NBASELINES']
Nblts = Ntimes * Nbls
Nfreqs = obs['N_FREQ']
Npols = len(vis_data.keys())
Nspws = 1
spw_array = [1]
vis_units = ['Jy']

lin_pol_order = ['xx','yy','xy','yx']
linear_pol_dict = dict(zip(lin_pol_order, np.arange(5,9)*-1))
pol_list = []
for pol in lin_pol_order:
    if pol in vis_data:
        pol_list.append(linear_pol_dict[pol])
polarization_array = np.asarray(pol_list)
print(polarization_array)

data_array = np.zeros((Nblts,Nspws,Nfreqs,Npols),dtype=np.complex_)
nsample_array = np.zeros((Nblts,Nspws,Nfreqs,Npols),dtype=np.complex_)
flag_array = np.zeros((Nblts,Nspws,Nfreqs,Npols),dtype=np.bool_)
for pol,vis in vis_data.items():
    pol_i = pol_list.index(linear_pol_dict[pol])
    data_array[:,0,:, pol_i] = vis
    flag_array[:,0,:, pol_i] = flag_data[pol] <= 0
    nsample_array[:,0,:, pol_i] = n.abs(flag_data[pol])

uvw_array = np.zeros((3,Nblts))
uvw_array[0,:] = params['UU'][0]
uvw_array[1,:] = params['VV'][0]
uvw_array[2,:] = params['WW'][0]

time_array = params['TIME'][0] # this isn't right. it needs to be added to a JD time, not sure which one (obs.jd0 and bl_info.jdate don't agree with eachother)

ant_1_array = bl_info['TILE_A'][0]
ant_2_array = bl_info['TILE_B'][0]

baseline_array = params['BASELINE_ARR'][0] # should really calculate this from the antenna arrays using bl_to_ij

freq_array = bl_info['FREQ'][0]
phase_center_ra = obs['PHASERA']
phase_center_dec = obs['PHASEDEC']

integration_time = obs['TIME_RES'] # why isn't this quite 2? Do we care?
channel_width = obs['FREQ_RES']

# # --- observation information ---
# object_name -- don't know if this exists in fhd save files
telescope_name = obs['INSTRUMENT'][0].decode("utf-8")
instrument = telescope_name
latitude = obs['LAT']
longitude = obs['LON']
altitude = obs['ALT']
dateobs = obs['JD0'] # check this!
# history -- don't know if anything should be put into this
phase_center_epoch = astrometry['EQUINOX'] # check this!

Nants = max(bl_info['TILE_A'][0]) # is this the best source?
antenna_names = bl_info['TILE_NAMES'][0]
antenna_indices = np.arange(Nants)


# # coordinate frame for antenna positions (eg 'ITRF' -also google ECEF)
# # NB: ECEF has x running through long=0 and z through the north pole
# 'xyz_telescope_frame'  : None,
# # coordinates of array center in meters in coordinate frame
# 'x_telescope'  : None,
# 'y_telescope'  : None,
# 'z_telescope'  : None,
# # array giving coordinates of antennas relative to
# # {x,y,z}_telescope in the same frame, (Nants,3)
# 'antenna_positions'  : None,

# # --- other stuff ---
# # the below are copied from AIPS memo 117, but could be revised to
# # merge with other sources of data.
# # when available they are populated. user beware?
# # Greenwich sidereal time at midnight on reference date
# 'GST0'  : None,
# 'RDate'  : None,  # date for which the GST0 or whatever... applies
# # earth's rotation rate in degrees per day
# # (might not be enough sigfigs)
# 'earth_omega'  : 360.985,
# 'DUT1'  : 0.0,        # DUT1 (google it) AIPS 117 calls it UT1UTC
# 'TIMESYS'  : 'UTC',   # We only support UTC


//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:19: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:20: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


[-5 -6]


//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:21: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


NameError: name 'n' is not defined

Index(['CODE_VERSION', 'INSTRUMENT', 'OBSNAME', 'DIMENSION', 'ELEMENTS',
       'NBASELINES', 'DFT_THRESHOLD', 'KPIX', 'DEGPIX', 'OBSAZ', 'OBSALT',
       'OBSRA', 'OBSDEC', 'ZENRA', 'ZENDEC', 'OBSX', 'OBSY', 'ZENX', 'ZENY',
       'PHASERA', 'PHASEDEC', 'ORIG_PHASERA', 'ORIG_PHASEDEC', 'N_POL',
       'N_TILE', 'N_TILE_FLAG', 'N_FREQ', 'N_FREQ_FLAG', 'N_TIME',
       'N_TIME_FLAG', 'N_VIS', 'N_VIS_IN', 'N_VIS_RAW', 'NF_VIS',
       'BEAM_INTEGRAL', 'POL_NAMES', 'JD0', 'MAX_BASELINE', 'MIN_BASELINE',
       'DELAYS', 'LON', 'LAT', 'ALT', 'FREQ_CENTER', 'FREQ_RES', 'TIME_RES',
       'ASTR', 'ALPHA', 'PFLAG', 'CAL', 'RESIDUAL', 'VIS_NOISE',
       'BASELINE_INFO', 'META_DATA', 'META_HDR', 'DEGRID_SPECTRAL_TERMS',
       'GRID_SPECTRAL_TERMS', 'GRID_INFO', 'HEALPIX'],
      dtype='object')